## Query Courses

### To load pickle file

In [1]:
from course import Course
import pickle

# Should take around a minute, pickle file is in Drive linked in README
with open("courses_directory.pkl", "rb") as f: 
    course_directory: dict[str, Course] = pickle.load(f) 

### Find Class

In [61]:
College: str = "CAS" # Three letter acronym like "CAS"
Department: str = "BI" # Two letter acronym like "BI"
Number: int = 203 # Three numbers like "203"

c: Course = course_directory[f"{College} {Department} {Number}"]

### Methods

#### Get Course Name, Code, and Attr

In [62]:
College, Department, Number = c.College, c.Department, c.Number

code: str = c.__repr__() # ex. CAS BI 203 

class_name: str = c.getCourseName() # ex. Cell Biology

#### Get PreReqs and CoReqs and Attr

In [63]:
pre_pres, co_reqs, leads_to = [c.UndergradReq + c.GradReq, \
                                c.UndergradCoReq + c.GradCoReq, c.Post]

# Returns clips with relivant info for course reqs
pre_req_text: dict[str, str] = c.getPreReqsText()

#### Hubs

In [64]:
Hubs: list[str] = c.getHubs()

#### Semesters Offered

In [65]:
semesters: list[str] = c.getSemesters()

#### tmi (to much information)

In [66]:
c.tmi() # returns str

'CAS BI 203 (Cell Biology) = R [CAS BI 108, CAS NE 102, CAS CH 102] C [CAS CH 203] '

#### Webpage and url

In [67]:
from bs4 import BeautifulSoup

website: BeautifulSoup = c.webpage

description: str = c.getDesc() # id='course-content' of course page

url: str = c.url

link: str = c.link # kwarg for c.__init__() for web scraping

valid_link: bool = c.valid_link # False if page not found

## Construct Course

In [ ]:
# Careful with this one, it will overwrite the pickle file and remove the Post list if you provide it the course_directory

name: str = "CAS BI 203"

c: Course = Course(name, link=None, cd={})

## Webscraping

In [ ]:
# Caution does a lot, broken have to declare BUCourseBranch class in same place as course_directory

from main import BUCourseBranch

save_branch = BUCourseBranch()

save_branch.run() # or save_branch.__repr__()

## Other

In [ ]:
# To JSON File

from dataclasses import dataclass

@dataclass
class json_comp():

    college: str # ex CAS
    department: str # ex BI
    number: str # ex 203

    name: str # ex Cell Biology

    desc: str # Blurb on webpage

    valid_link: bool # if webpage exists

    uprereqs: list[str] # Undergrad Pre reqs
    gprereqs: list[str] # Grad Pre Reqs

    uprereqstext: str
    gprereqstext: str

    ucoreqs: list[str] # Undergrad Co reqs
    gcoreqs: list[str] # Grad Co Reqs

    ucoreqstext: str
    gcoreqstext: str

    post: list[str] # Classes lead to

    hubs: list[str] # Hubs satisifed

    semesters: list[str] # ['FALL', 'SPNG']
  

to_json: dict[str, json_comp] = {}

for name, _class in zip(course_directory.keys(), course_directory.values()):

    to_json.__setitem__(name, json_comp(
        college=_class.College,
        department=_class.Department,
        number=_class.Number,
        name=_class.getCourseName(),
        desc=_class.getDesc(),
        valid_link=_class.valid_link,
        uprereqs=[c.__repr__() for c in _class.UndergradReq],
        gprereqs=[c.__repr__() for c in _class.GradReq],
        ucoreqs=[c.__repr__() for c in _class.UndergradCoReq],
        gcoreqs=[c.__repr__() for c in _class.GradCoReq],
        uprereqstext=_class.getPreReqsText()['upr'] if 'upr' in _class.getPreReqsText() else "",
        gprereqstext=_class.getPreReqsText()['gpr'] if 'gpr' in _class.getPreReqsText() else "",
        ucoreqstext=_class.getPreReqsText()['ucr'] if 'ucr' in _class.getPreReqsText() else "",
        gcoreqstext=_class.getPreReqsText()['gcr'] if 'gcr' in _class.getPreReqsText() else "",
        post=[c.__repr__() for c in _class.Post],
        hubs=_class.getHubs(),
        semesters=_class.getSemesters()
    ).__dict__)

In [79]:
class dummy():

    def __init__(self, c):
        self.c = c

    def __repr__(self):
        return self.c

    def getHubs(self):

        print(self.c)

        return []
    
    def getSemesters(self):

        print(self.c)

        return []

In [ ]:
def load_courses() -> list[Course]:
    courses: list[str] = []

    masters = ['MF', 'MS']

    with open('my_courses.txt', 'r') as mc:

        text = mc.read().split('\n')

        for semester in text:

            name, info = semester.split(' ? ')

            cs = [course_directory.get(' '.join([i for i in c.split(' ') if i != '']), dummy(' '.join([i for i in c.split(' ') if i != '']))) for c in info.split('|')]

            semester = None
            
            match name[[i for i, a, b in zip(range(name.__len__()), name, name.upper()) if a == b][1]:]:
                
                case 'F':

                    semester = 'FALL'

                case 'S':

                    semester = 'SPRG'

                case 'Sum':

                    semester =  'SUMMER'

                case _:

                    raise ValueError(f'semester not found')

            if semester != 'SUMMER':

                for c in cs:

                    course_sem = c.getSemesters()

                    if not course_sem:
                        
                        print(f'{colors["yellow"]}{c.__repr__()} unclear {colors["white"]}')

                    elif semester not in course_sem:

                        print(f'{colors["red"]}{c.__repr__()} not in {semester}, {c.getSemesters()}{colors["white"]}')
                    
                    else:

                        print(f'{colors["green"]}{c.__repr__()} in {semester}, {c.getSemesters()}{colors["white"]}')


            if '*' not in info and name.replace(' ', '') not in masters:

                # f'{Col} {Dep} {Num}' for Col, Dep, Num

                courses.extend(cs)

    return courses

courses = load_courses()

In [ ]:
import numpy as np

from collections.abc import Callable

from typing import Tuple, Callable, Union

from colors import colors

def find_courses_with(hub: str) -> list[Course]:
    return [c for c in course_directory.values() if hub in c.getHubs()]

def course_reports(cs: list[Course]) -> None:
    print(*[f'{c.__repr__()}: {c.getCourseName()} -> {c.getSemesters()} | {c.all_reqs} :: {c.getCredits()}' for c in cs], sep='\n')

def hubs_report(cs: list[Course]) -> dict[str, int]:

    total = np.unique(sum([c.getHubs() for c in cs], []), return_counts=True)

    print(*[f'{name}:{num}' for name, num in zip(*total, [])], sep='\n')

    hubs = {}

    [hubs.__setitem__(name, num) for name, num in zip(*total)]

    return hubs


def check_hub(hub: Union[list[str], str], num: Union[list[int], int]) -> Callable:

    def check(user_hubs: dict[str, int]) -> bool:

        if isinstance(hub, list):

            return sum([h in user_hubs and int(user_hubs[h]) >= int(num) for h, num in zip(hub, num)]).__bool__()
        
        elif isinstance(hub, str):

            return hub in user_hubs and int(user_hubs[hub]) >= int(num)

        else:

            raise TypeError(f"hub arg should be of type str or list not {type(hub)}")
        
    return check

def load_requirements() -> list[str]:

    with open('hub_requirements.txt', 'r') as hr:

        return hr.read().split('\n')

def check_requirements() -> bool:

    checks = [c for c in [l.split(':') if '||' not in l else np.array([o.split(':') for o in l.split(' || ')]).T.tolist() for l in load_requirements()]]

    print(checks)

    user_hubs = hubs_report(load_courses())

    for check in checks:

        output: str = f'{":".join([*check])}<={user_hubs.get(check[0], 0)}' if isinstance(check[0], str) else ' || '.join([f'{":".join([*ch])}<={user_hubs.get(ch[0], 0)}' for ch in np.array(check).T.tolist()])

        if check_hub(*check)(user_hubs):

            print(f'{colors["green"]}{output}{colors["white"]}')

        else:

            print(f'{colors["red"]}{output}{colors["white"]}')


check_requirements()
